In [1]:
# %load '/Users/virginiarutten/Documents/code/utils/template.py'
from __future__ import print_function
import matplotlib.pyplot as pl
import pyqtgraph as pg
import matplotlib as mpl
import seaborn as sns
import numpy as np
from imp import reload
import os, sys
import cv2
from PyQt5 import QtCore
from imageio import imwrite
np.set_printoptions(precision=3, suppress=1)
sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})
%matplotlib inline
%gui qt

In [2]:
from MyImageViewer import MyImageViewer as im
from MyCellDetector import MyCellDetector as mcd

In [34]:
def rload(module):
    reload(sys.modules[module.__name__])
def scale(im):
    max_ = im.max()
    return (im/max_*255).astype('uint8')
def saturate(frame, max_ = 20):
    fr = frame.copy()
    fr[fr>max_] = max_
    return fr.astype('uint8')
def write2im(im):
    im_wText = cv2.putText(im.swapaxes(0,1).copy(), "# cells: {}".format(keypoints[0]['num']),\
            (10,100), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 255), 2)
    return im_wText.swapaxes(0,1)

#### define image path

In [35]:
file_path = './images/Cellcounting_Example.jpeg'
results_path = './results/'

#### create our cell detector

In [14]:
C = mcd()

#### load image - convert to grayscale - blur

In [15]:
frame = C.read_file(file_path)
gray = C.gray_frame(frame)
blur = C.blur_frame(gray, kernel = (3,3), filterType = 'median')

In [54]:
fr_im = im(frame)
bl_im = im(scale(saturate(blur, 50)))

#### save images

In [56]:
imwrite(results_path + 'orginal.png', frame.swapaxes(0,1))
imwrite(results_path + 'blur.png', scale(saturate(blur.swapaxes(0,1), 50)))

#### find threshold or set manually

In [57]:
perc= 99
pVal_l = C.get_percentile(blur, perc = perc)
print('{0}th percentile: {1}'.format(perc, pVal_l))
pVal_l = 4

99th percentile: 1.0


#### morphological operators: mask, erode and dilate

In [58]:
mask, masked =  C.mask_frame(blur, pVal_l = pVal_l)

In [59]:
imwrite(results_path + 'raw_mask_' +str(pVal_l) + '.png', scale(mask).swapaxes(0,1))
imwrite(results_path + 'masked_' +str(pVal_l) + '.png', scale(masked).swapaxes(0,1))

In [60]:
### this erodes/dilated the mask itself
eMask = C.erode_frame(mask, ker = 5)
dMask = C.dilate_frame(eMask, ker = 5)

### this erodes/dilated the masked image
eFr = C.erode_frame(masked, ker = 5)
dFr = C.dilate_frame(eFr, ker = 5)

#### visualise

In [61]:
m_im = im(mask, 'mask')
e_im = im(eMask, 'eroded')
d_im = im(dMask, 'eroded + dilated')

In [62]:
mF_im = im(masked, 'mask')
eF_im = im(eFr, 'eroded')
dF_im = im(dFr, 'eroded + dilated')

In [63]:
# rload(mcd)
# from MyCellDetector import MyCellDetector as mcd
# C = mcd()

#### detect contours and filter on area size
(we can also filter on convextity/other criteria if you want)

In [64]:
area_mn = 4
area_mx = 1e10

In [65]:
### finds keypoints of eroded + dilated masked image
keypoints = C.countour_frame(dFr, area_mn = area_mn, area_mx = area_mx, keypoints = {})
### finds keypoints of eroded + dilated mask
keypointsMask = C.countour_frame(dMask, area_mn = area_mn, area_mx = area_mx, keypoints = {})

#### overlay

In [66]:
dispImCont, dispImCenter, dispImBox = C.print_keypoints(keypoints[0],frame, col = (0,100,0))
dispMaskCont, dispMaskCenter, dispMaskBox = C.print_keypoints(keypoints[0],mask*255,  col = (255,0,0))

In [67]:
dispImCenter_ = write2im(dispImCenter)
dispMaskCenter_ = write2im(dispMaskCenter)

In [68]:
im(scale(saturate(dispImCenter_)))

In [69]:
ims = [dispImCont, dispImCenter_, dispImBox, dispMaskCont, dispMaskCenter_, dispMaskBox]

In [70]:
scaled_ims = [scale(saturate(my_image)) for my_image in ims]

In [71]:
names = ['detection_wContours_', 'detection_wCenters_', 'detection_wBox_',\
         'mask_wContours_', 'mask_wCenters_' ,  'mask_wBox_']

In [72]:
tmp = [imwrite(results_path + names[ind] +str(pVal_l) + '.png', im_.swapaxes(0,1))  for ind, im_ in enumerate(scaled_ims)]

In [70]:
t2 = im(saturate(dispImCenter_))

In [71]:
t1 = im(saturate(dispImCont))
t3 = im(saturate(dispImBox))

In [72]:
t1 = im(dispMaskCont)
t2 = im(dispMaskCenter)
t3 = im(dispMaskBox)